In [3]:
import numpy as np
from collections import defaultdict
import random
from numpy import random
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## old setup
$$
\begin{equation*}
\begin{array}{lll}
&\displaystyle \min_{\substack{
u^{\text{dis}}_{t},\ u^{\text{chr}}_{t},p^{\text{dis}}_{t},\ p^{\text{chr}}_{t},x^{\text{chiller}}_{j,t},\ x^{\text{tower}}_{j,t} \\
T_{i,t}^{\text{Zone}},\ T_{i,t+1}^{\text{Zone}} ,T_{i,t}^{\text{sup}},\ v^{\text{vent}}_{t} \\}} 
\sum_{t=0}^{T}p^{e,g}_{t} e^{\text{dc,in}}_{t} & \\
s.t.\quad &e^{\text{dc,in}}_{t} = E^{\text{HVAC}}_t + E^{\text{Se}}_{t} +\Delta E^{\text{B}}_{t} - E^{\text{Solar}}_{t} & \forall  t \in \mathbf{T}\\
&e^{\text{dc,in}}_{t} \geq 0 & \forall t \in \mathbf{T}\\
& \Delta E^{\text{B}}_{t} = p^{\text{chr}}_{t}\eta^{\text{chr}} - p^{\text{dis}}_{t} \cdot (\eta^{\text{dis}})^{-1} & \forall  t \in \mathbf{T}\\ 
& E^{\text{B,state}}_{t+1} = E^{\text{B,state}}_{t} + \Delta E^{\text{B}}_{t} & \forall  t \in \mathbf{T} \\
& \underline{\xi^{\text{B}}} \leq E^{\text{B,state}}_{t} \leq \overline{\xi^{\text{B}}} & \forall  t \in \mathbf{T} \\
& p^{\text{chr}}_{t} \leq \overline{p^{\text{chr}}_{t}} \cdot u^{\text{chr}}_{t} & \forall  t \in \mathbf{T}\\
& p^{\text{dis}}_{t} \leq \overline{p^{\text{dis}}_{t}} \cdot u^{\text{dis}}_{t} & \forall  t \in \mathbf{T}\\
& u^{\text{chr}}_{t} + u^{\text{dis}}_{t} \leq 1 & \forall  t \in \mathbf{T}\\
&T^{\text{Zone}}_{i,t+1} = T^{\text{Zone}}_{i,t} 
+ \displaystyle\sum_{i^{\prime} \in  \mathcal{N}\left(i\right)} 
\left(\frac{T_{i^{\prime},t}^{\text{Zone}} - T_{i,t}^{\text{Zone}}}
{C^{\text{heat}}_{i}R^{\text{Zone}}_{i^{\prime}i}}\right)
+ \frac{\dot{m}_{i} c_{a}\left(T_{i,t}^{\text{sup}} - T_{i,t}^{\text{Zone}}\right) 
+ \theta_{i,t}}{C^{\text{heat}}_{i}} & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T}\\
&T_{i,t}^{-} \leq T^{\text{Zone}}_{i,t} \leq T_{i,t}^{+}, & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T} \\
&v^{\text{vent}}_t + v^{\text{out}}_t \geq \underline{v}^{\text{vent}}_t & \forall  t \in \mathbf{T}\\
&L^{\text{heat}}_{t} = \left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{chiller}}} x^{\text{chiller}}_{j,t}m^{\text{chiller}}_{j,t} \geq \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{tower}}}{x^{\text{tower}}_{j,t}m^{\text{tower}}_{j,t}} \geq \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p }
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot  c^{\text{water}}_p }  & \forall t \in \mathbf{T}\\
&e^{\text{vent}}_{t}=\beta^{\text{vent}}_{0}+\beta^{\text{vent}}_{1}\left(v^{\text{vent}}_t-\underline{v}^{\text{vent}}\right) 
& \forall t \in \mathbf{T}\\
&e^{\text{chiller}}_{t}=\displaystyle\sum_{j \in \mathbf{J}^{\text{chiller}}}x^{\text{chiller}}_{j,t}\left(\beta^{\text{chiller}}_{0,j}
 + \beta^{\text{chiller}}_{1,j}m^{\text{chiller}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&e^{\text {pump }}_{t}=\beta^{\text{pump}}_{0}+\beta^{\text{pump}}_{1} \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \left(\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }{\chi_{i}T^{\text{Zone}}_{i,t}}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&e^{\text{tower}}_{t} = \displaystyle\sum_{j \in \mathbf{J}^{\text{tower}}}x^{\text{tower}}_{j,t} \left(\beta^{\text{tower}}_{0,j} + \beta^{\text{tower}}_{1,j}m^{\text{tower}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&E^{\text{HVAC}}_{t} =  e^{\text{sup}}_{t} + e^{\text{vent}}_{t}+ e^{\text{chiller}}_{t} + e^{\text{pump}}_{t} + e^{\text{tower}}_{t} & \forall  t \in \mathbf{T}\\
\end{array}
\end{equation*}
$$

## New Setup

$$
\begin{equation*}
\begin{array}{lll}
&\displaystyle \min_{\substack{
u^{\text{dis}}_{t},\ u^{\text{chr}}_{t},p^{\text{dis}}_{t},\ p^{\text{chr}}_{t},x^{\text{chiller}}_{j,t},\ x^{\text{tower}}_{j,t} \\
T_{i,t}^{\text{Zone}},\ T_{i,t+1}^{\text{Zone}} ,T_{i,t}^{\text{sup}},\ v^{\text{vent}}_{t} \\}} 
\sum_{t=0}^{T}p^{e,g}_{t} e^{\text{dc,in}}_{t} & \\
s.t.\quad &e^{\text{dc,in}}_{t} = E^{\text{HVAC}}_t + E^{\text{Se}}_{t} +\Delta E^{\text{B}}_{t} - E^{\text{Solar}}_{t} & \forall  t \in \mathbf{T}\\
&e^{\text{dc,in}}_{t} \geq 0 & \forall t \in \mathbf{T}\\
& \Delta E^{\text{B}}_{t} = p^{\text{chr}}_{t}\eta^{\text{chr}} - p^{\text{dis}}_{t} \cdot (\eta^{\text{dis}})^{-1} & \forall  t \in \mathbf{T}\\ 
& E^{\text{B,state}}_{t+1} = E^{\text{B,state}}_{t} + \Delta E^{\text{B}}_{t} & \forall  t \in \mathbf{T} \\
& \underline{\xi^{\text{B}}} \leq E^{\text{B,state}}_{t} \leq \overline{\xi^{\text{B}}} & \forall  t \in \mathbf{T} \\
& p^{\text{chr}}_{t} \leq \overline{p^{\text{chr}}_{t}} \cdot u^{\text{chr}}_{t} & \forall  t \in \mathbf{T}\\
& p^{\text{dis}}_{t} \leq \overline{p^{\text{dis}}_{t}} \cdot u^{\text{dis}}_{t} & \forall  t \in \mathbf{T}\\
& u^{\text{chr}}_{t} + u^{\text{dis}}_{t} \leq 1 & \forall  t \in \mathbf{T}\\
&T^{\text{Zone}}_{i,t+1} = T^{\text{Zone}}_{i,t} 
+ \displaystyle\sum_{i^{\prime} \in  \mathcal{N}\left(i\right)} 
\left(\frac{T_{i^{\prime},t}^{\text{Zone}} - T_{i,t}^{\text{Zone}}}
{C^{\text{heat}}_{i}R^{\text{Zone}}_{i^{\prime}i}}\right)
+ \frac{\dot{m}_{i} c_{a}\left(T_{i,t}^{\text{sup}} - T_{i,t}^{\text{Zone}}\right) 
+ \theta_{i,t}}{C^{\text{heat}}_{i}} & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T}\\
&T_{i,t}^{-} \leq T^{\text{Zone}}_{i,t} \leq T_{i,t}^{+}, & \forall i \in \mathbf{I}^{\text{Zone}}, t \in \mathbf{T} \\
&v^{\text{vent}}_t + v^{\text{out}}_t \geq \underline{v}^{\text{vent}}_t & \forall  t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{chiller}}} x^{\text{chiller}}_{j,t}m^{\text{chiller}}_{j,t} \geq \frac{\left(T^{\text{out}}_{t}-\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}}}\chi_{i}T^{\text{sup}}_{i,t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&\displaystyle \sum_{j \in \mathbf{J}^{\text{tower}}}{x^{\text{tower}}_{j,t}m^{\text{tower}}_{j,t}} \geq \frac{\left(T^{\text{out}}_{t}-\displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}}}\chi_{i}T^{\text{sup}}_{i,t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p }
{\left(T^{\text{conwr}}_{t}-T^{\text{conws}}_{t}\right)\cdot  c^{\text{water}}_p }  & \forall t \in \mathbf{T}\\
&e^{\text{vent}}_{t}=\beta^{\text{vent}}_{0}+\beta^{\text{vent}}_{1}\left(v^{\text{vent}}_t-\underline{v}^{\text{vent}}\right) 
& \forall t \in \mathbf{T}\\
&e^{\text{chiller}}_{t}=\displaystyle\sum_{j \in \mathbf{J}^{\text{chiller}}}x^{\text{chiller}}_{j,t}\left(\beta^{\text{chiller}}_{0,j}
 + \beta^{\text{chiller}}_{1,j}m^{\text{chiller}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&e^{\text {pump }}_{t}=\beta^{\text{pump}}_{0}+\beta^{\text{pump}}_{1} \frac{\left(T^{\text{out}}_{t}-T^{\text{sup}}_{t}\right)\cdot v^{\text{out}}_{t} c^{\text{air}}_p + \displaystyle \sum_{i \in \mathbf{I}^{\text{Zone}} }\chi_{i}\left(T^{\text{Zone}}_{i,t}-T^{\text{sup}}_{t}\right)\cdot \left(v^{\text{sup}} - v^{\text{out}}_t\right) c^{\text{air}}_p}
{\left(T^{\text{chwr}}_{t}-T^{\text{chws}}_{t}\right)\cdot c^{\text{water}}_p}  & \forall t \in \mathbf{T}\\
&e^{\text{tower}}_{t} = \displaystyle\sum_{j \in \mathbf{J}^{\text{tower}}}x^{\text{tower}}_{j,t} \left(\beta^{\text{tower}}_{0,j} + \beta^{\text{tower}}_{1,j}m^{\text{tower}}_{j,t}\right) & \forall t \in \mathbf{T}\\
&E^{\text{HVAC}}_{t} =  e^{\text{sup}}_{t} + e^{\text{vent}}_{t}+ e^{\text{chiller}}_{t} + e^{\text{pump}}_{t} + e^{\text{tower}}_{t} & \forall  t \in \mathbf{T}\\
\end{array}
\end{equation*}
$$


In [27]:
# Total time interval
T = 4

# Total Data center zones
I_zone_wide = 3
I_zone_length = 3
I_zone =  I_zone_wide * I_zone_length

# time interval length
delta_t = 15 # minutes

# Real time grid electricity price
p_eg = np.arange(1, T + 1, 1, dtype=int)*200

# sever electricity consumpsion
e_server = 60 * np.ones(T)  #kWh

# Solar electricity production
e_solar = 0 * np.ones(T)      #kWh

# Batter electricity charging upper bound
char_b_upperbound = 30

# Batter electricity discharging upper bound
dis_b_upperbound = 50

# Battery electricity capacity
Battery_capacity = 150 #  unit kWh

# Battery initial energy
Battery_capacity_init = 0.5 * Battery_capacity #  unit kWh

# Battery electricity capacity uppper and lower alert
Battery_capacity_upbound = 0.85 * Battery_capacity
Battery_capacity_lowbound = 0.15 * Battery_capacity 

#battery charging efficiency
eta_char_b = 0.9

#battery discharging efficiency
eta_dischar_b = 0.85

#zone tempreture initialization celcius or fahrenheit
T_initial_upperbound = 25
T_initial_lowerbound = 33 
random_choice = 0  
if random_choice == 0 :
    T_zone_initial =np.array([[31, 30, 29],[32, 26, 26],[30, 24, 27]])
else:
    T_zone_initial = np.random.randint(25, 33, (I_zone_wide, I_zone_length))
    
    
#Zone space 
if random_choice == 0 :
    Zone_space = np.array([[1020,  950,  950],[1010, 1060, 1040],[ 900, 1050,  930]])
else:
    Zone_space = np.random.randint(900, 1100, (I_zone_wide, I_zone_length))
    
DC_height = 6 # meters


# zone air
## air thermal resistance value (0.4-0.6)
## https://www-sciencedirect-com.ezproxy.lib.uh.edu/topics/engineering/thermal-resistance
## https://fscdn.rohm.com/en/products/databook/applinote/common/basics_of_thermal_resistance_and_heat_dissipation_an-e.pdf
## http://www.sfu.ca/phys/346/121/lecture_notes/lecture33_heat_loss.pdf

R_air_ij = 0.062  #unit m.K/W

# air heat capacity in room i
#C_air =  718 * 1.225 * 100 * np.ones(I_zone) # unit J/K  rho = 1.225 kg/m3

# air mass flow into the room i
m_air_in =   4 * np.kron(v_sup.reshape(1,-1)  , np.ones((I_zone,1)) ) * 1.2  # unit m2 * m/s * Kg/m^3 * s

# specific heat capacity of air in room i 
c_s_air = 0.718 #unit [KJ/( kg · K )]

# internal heat generation of server radiated to room i
theta_heat = 1 * np.kron(e_server.reshape(1,-1) , np.ones((I_zone,1))) * 3600 / (I_zone)  #unit W/K

# outside wind speed
v_out = np.ones(T) * 0.5 * delta_t * 60 # unit m/s

# AC wind speed
v_sup = np.ones(T) * 1 * delta_t * 60 # unit m/s

# AC wind power
beta_sup = 5 # unit kWh/(m/s)
e_sup = beta_sup * v_sup # unit kWh

# ventilation air flow rate lower bound
v_vent_lowerbound = np.ones(T) * 0.8 # unit m/s

# heat capacity of air.
C_air_room =  0.718 * 1.2 * Zone_space * DC_height  # unit [KJ/( kg · K )] *  kg/m3 * m3  = KJ/K

# heat capacity of water.
C_water =  4.182  # unit [KJ/( kg · K )]

# Outside air temperature
T_out = np.ones(T) * 32 # unit K

# Room air temperature upper bound
T_datacenter_upperbound = np.ones((I_zone,T)) * 24

# Room air temperature lower bound
T_datacenter_lowerbound = np.ones((I_zone,T)) * 15

# Room AC temperature upper bound
T_AC_upperbound = np.ones((I_zone,T)) * 26

# Room AC temperature lower bound
T_AC_lowerbound = np.ones((I_zone,T)) * 10

# chiller water temperature supply
T_chiller_water_supply = np.ones(T) * 20

# chiller water temperature return
T_chiller_water_return = np.ones(T) * 30

# condense water temperature supply
T_tower_water_supply = T_out

# condense water temperature return
T_tower_water_return = np.ones(T) * 45

# ventilation air flow rate start up power
v_vent_startup = 0.8 * delta_t  # unit m/s
beta_vent_0 = 100 # unit kJ
beta_vent_1 = 10 # unit kJ s/m

# pump water flow rate power
beta_pump_0 = 100 # unit kJ
beta_pump_1 = 10 # unit kJ/(kg)

# chiller amount
J_chiller = 5

# tower amount 
J_tower = 8

# chiller rated water flow rate
M_chiller_rated = 30 * np.ones(J_chiller) # unit kg/t

# tower rated water flow rate
M_tower_rated = 40 * np.ones(J_tower) # unit kg/t    

# chiller rated power
beta_chiller_0 = 4 * np.ones(J_chiller) # unit kWh
beta_chiller_1 = 7 * np.ones(J_chiller) # unit kWh/(kg)

# tower rated power
beta_tower_0 = 6 * np.ones(J_tower) # unit kWh
beta_tower_1 = 5 * np.ones(J_tower) # unit kWh/(kg)

# weight of each zone in data center
Chi = np.ones(I_zone)*(1/I_zone)

#adjacent zone map (default)
#4|5|6|
#1|2|3|
adjacent_zone_map = np.zeros((I_zone,I_zone))
for zone in range(I_zone):
    zone_x  = zone // I_zone_length
    zone_y = zone % I_zone_length
    
    adjacent_zone_x = np.array([zone_x-1, zone_x+1])
    adjacent_zone_y = np.array([zone_y-1, zone_y+1])
    
    adjacent_zone_x_update = adjacent_zone_x[ (adjacent_zone_x >= 0) & (adjacent_zone_x < I_zone_wide) ]
    adjacent_zone_y_update = adjacent_zone_y[ (adjacent_zone_y >= 0) & (adjacent_zone_y < I_zone_length) ]
    
   # print(adjacent_zone_x, adjacent_zone_y)
    
    adjacent_zone_map[zone, adjacent_zone_x_update * I_zone_length + zone_y] = 1
    adjacent_zone_map[zone, zone_x * I_zone_length + + adjacent_zone_y_update] = 1

#print(adjacent_zone_map)

In [26]:
m_air_in

array([[8640., 8640., 8640., 8640.],
       [8640., 8640., 8640., 8640.],
       [8640., 8640., 8640., 8640.],
       [8640., 8640., 8640., 8640.],
       [8640., 8640., 8640., 8640.],
       [8640., 8640., 8640., 8640.],
       [8640., 8640., 8640., 8640.],
       [8640., 8640., 8640., 8640.],
       [8640., 8640., 8640., 8640.]])

In [24]:
1.2 * Zone_space * DC_height

array([[7344., 6840., 6840.],
       [7272., 7632., 7488.],
       [6480., 7560., 6696.]])

In [25]:
C_air_room

array([[5272.992, 4911.12 , 4911.12 ],
       [5221.296, 5479.776, 5376.384],
       [4652.64 , 5428.08 , 4807.728]])